In [14]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb

In [15]:
minio_client = conectar_minio(
    endpoint='localhost:9000',
    access_key='crosero',
    secret_key='12345678',
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [16]:
def extract_pdf_content(pdf_document):
    """
    Extrae contenido específico de un archivo PDF ya abierto.

    Args:
        pdf_document (fitz.Document): Documento PDF abierto con PyMuPDF.

    Returns:
        dict: Diccionario con la información extraída.
    """
    extracted_data = {
        "Numero de Informe": None,
        "Nombre del Paciente": None,
        "Identificacion": None,
        "Edad": None,
        "Telefono": None,
        "Fecha de Toma de Muestra": None,
        "Fecha de Ingreso": None,
        "Fecha de Informe": None,
        "Entidad": None,
        "EPS": None,
        "Servicio": None,
        "Muestra Remitida": None,
        "Descripcion Macroscopica": None,
        "Descripcion Microscopica": None,
        "Diagnostico": [],
        "Comentario": None
    }

    try:
        # Leer cada página del PDF
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text = page.get_text("text")
            cleaned_text = " ".join(text.split())
            
            # Extraer información utilizando búsquedas
            if "INFORME NÚMERO:" in cleaned_text:
                extracted_data["Numero de Informe"] = cleaned_text.split("INFORME NÚMERO:")[1].split()[0].strip()
            if "NOMBRE:" in cleaned_text:
                extracted_data["Nombre del Paciente"] = cleaned_text.split("NOMBRE:")[1].split("IDENTIFICACIÓN:")[0].strip()
            if "IDENTIFICACIÓN:" in cleaned_text:
                extracted_data["Identificacion"] = cleaned_text.split("IDENTIFICACIÓN:")[1].split("EDAD:")[0].strip()
            if "EDAD:" in cleaned_text:
                extracted_data["Edad"] = cleaned_text.split("EDAD:")[1].split()[0].strip()
            if "TELÉFONO:" in cleaned_text:
                extracted_data["Telefono"] = cleaned_text.split("TELÉFONO:")[1].split()[0].strip()
            if "FECHA TOMA DE MUESTRA:" in cleaned_text:
                extracted_data["Fecha de Toma de Muestra"] = cleaned_text.split("FECHA TOMA DE MUESTRA:")[1].split()[0].strip()
            if "FECHA DE INGRESO:" in cleaned_text:
                extracted_data["Fecha de Ingreso"] = cleaned_text.split("FECHA DE INGRESO:")[1].split("FECHA DE INFORME:")[0].strip()
            if "FECHA DE INFORME:" in cleaned_text:
                extracted_data["Fecha de Informe"] = cleaned_text.split("FECHA DE INFORME:")[1].split()[0].strip()
            if "ENTIDAD:" in cleaned_text:
                extracted_data["Entidad"] = cleaned_text.split("ENTIDAD:")[1].split("EPS")[0].strip()
            if "EPS:" in cleaned_text:
                extracted_data["EPS"] = cleaned_text.split("EPS:")[1].split("SERVICIO")[0].strip()
            if "SERVICIO:" in cleaned_text:
                extracted_data["Servicio"] = cleaned_text.split("SERVICIO:")[1].split()[0].strip()
            if "MUESTRA REMITIDA:" in cleaned_text:
                extracted_data["Muestra Remitida"] = cleaned_text.split("MUESTRA REMITIDA:")[1].split()[0].strip()
            if "DESCRIPCION MACROSCOPICA:" in cleaned_text:
                extracted_data["Descripcion Macroscopica"] = cleaned_text.split("DESCRIPCION MACROSCOPICA:")[1].split("DESCRIPCION MICROSCOPICA:")[0].strip()
            if "DESCRIPCION MICROSCOPICA:" in cleaned_text:
                extracted_data["Descripcion Microscopica"] = cleaned_text.split("DESCRIPCION MICROSCOPICA:")[1].split("DIAGNOSTICO:")[0].strip()
            if "DIAGNOSTICO:" in cleaned_text:
                diagnostic_section = cleaned_text.split("DIAGNOSTICO:")[1].strip().split("\n")
                extracted_data["Diagnostico"] = [line.strip() for line in diagnostic_section if line.strip()]
            if "COMENTARIO:" in cleaned_text:
                extracted_data["Comentario"] = cleaned_text.split("COMENTARIO:")[1].strip()

    except Exception as e:
        print(f"Error al extraer datos del PDF: {e}")

    return extracted_data

In [17]:
import pandas as pd
import fitz  # PyMuPDF

def process_all_pdfs_in_minio(minio_client, bucket_name, folder_path):
    """
    Procesa todos los archivos PDF en una ruta específica de un bucket MinIO.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde están los archivos
    folder_path : str
        Ruta dentro del bucket (ej: 'informes/2023/')
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los datos extraídos de todos los PDFs
    """
    results = []
    
    try:
        # Asegurar que la ruta termine con '/'
        if not folder_path.endswith('/'):
            folder_path += '/'
        
        # Listar objetos en la ruta especificada
        objects = minio_client.list_objects(bucket_name, prefix=folder_path, recursive=True)
        
        for obj in objects:
            if obj.object_name.lower().endswith('.pdf'):
                print(f"Procesando archivo: {obj.object_name}")
                pdf_data = None
                pdf_document = None
                
                try:
                    # Obtener el objeto PDF
                    response = minio_client.get_object(bucket_name, obj.object_name)
                    pdf_data = BytesIO(response.read())
                    
                    # Abrir el PDF
                    pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
                    
                    # Extraer contenido (asumiendo que tienes esta función)
                    content = extract_pdf_content(pdf_document)
                    
                    # Agregar los datos al resultado
                    results.append({
                        "Archivo": obj.object_name.split('/')[-1],  # Solo el nombre del archivo
                        "Ruta Completa": obj.object_name,
                        "Numero de Informe": content.get("Numero de Informe"),
                        "Nombre del Paciente": content.get("Nombre del Paciente"),
                        "Identificacion": content.get("Identificacion"),
                        "Edad": content.get("Edad"),
                        "Telefono": content.get("Telefono"),
                        "Fecha de Toma de Muestra": content.get("Fecha de Toma de Muestra"),
                        "Fecha de Ingreso": content.get("Fecha de Ingreso"),
                        "Fecha de Informe": content.get("Fecha de Informe"),
                        "Entidad": content.get("Entidad"),
                        "EPS": content.get("EPS"),
                        "Servicio": content.get("Servicio"),
                        "Muestra Remitida": content.get("Muestra Remitida"),
                        "Descripcion Macroscopica": content.get("Descripcion Macroscopica"),
                        "Descripcion Microscopica": content.get("Descripcion Microscopica"),
                        "Diagnostico": content.get("Diagnostico"),
                        "Comentario": content.get("Comentario"),
                    })
                    
                except Exception as e:
                    print(f"Error procesando {obj.object_name}: {e}")
                finally:
                    if pdf_document is not None:
                        pdf_document.close()
                    if 'response' in locals():
                        response.close()
                        response.release_conn()
    
    except S3Error as e:
        print(f"Error al acceder al bucket {bucket_name}: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")
    
    # Convertir resultados en un DataFrame
    return pd.DataFrame(results)

In [18]:
def clean_entidad_field(entidad):
    """
    Limpia el campo `Entidad` eliminando cualquier información relacionada con
    "FECHA DE INGRESO" y "FECHA DE INFORME".

    Args:
        entidad (str): Texto del campo `Entidad` a limpiar.

    Returns:
        str: Texto limpio del campo `Entidad`.
    """
    if not isinstance(entidad, str):
        return entidad  # Si no es una cadena, devolver tal cual
    
    # Expresión regular para encontrar y eliminar las fechas
    cleaned_entidad = re.sub(r'FECHA DE INGRESO:.*?(FECHA DE INFORME:.*)?$', '', entidad, flags=re.IGNORECASE).strip()
    return cleaned_entidad



In [19]:
def clean_special_characters(text):
    """
    Limpia caracteres especiales y no deseados en un texto o lista de textos.

    Args:
        text (str | list): Texto o lista de textos a limpiar.

    Returns:
        str: Texto limpio.
    """
    if isinstance(text, list):
        # Si es una lista, convertirla en una sola cadena separada por espacios
        text = ' '.join(text)
    
    if isinstance(text, str):
        # Reemplazar \uf0d8 con viñeta estándar
        text = text.replace('\uf0d8', '•')
        # Eliminar corchetes y otros caracteres especiales
        text = re.sub(r'[\[\]\"]', '', text)
        # Eliminar caracteres no ASCII imprimibles
        text = re.sub(r'[^\x20-\x7E]', '', text)
        return text.strip()  # Eliminar espacios en blanco adicionales

    return text  # Si no es cadena ni lista, devolver sin cambios


In [20]:
def process_identificacion_column(df):
    """
    Divide la columna 'Identificacion' en 'TipoIdentificacion' y 'NumeroIdentificacion',
    renombra 'Identificacion' a 'LlavePaciente', y elimina los espacios en 'LlavePaciente'.

    Args:
        df (pd.DataFrame): DataFrame con la columna 'Identificacion'.

    Returns:
        pd.DataFrame: DataFrame modificado con las nuevas columnas.
    """
    # Separar los dos primeros caracteres como 'TipoIdentificacion'
    df['TipoIdentificacion'] = df['Identificacion'].str[:2].str.strip()
    
    # Separar el resto del texto como 'NumeroIdentificacion'
    df['NumeroIdentificacion'] = df['Identificacion'].str[3:].str.strip()
    
    # Renombrar la columna 'Identificacion' a 'LlavePaciente'
    df.rename(columns={'Identificacion': 'id_paciente'}, inplace=True)
    
    # Eliminar espacios en 'LlavePaciente' y convertir a minúsculas
    df['id_paciente'] = df['id_paciente'].str.replace(r'\s+', '', regex=True).str.upper()

    
    return df

In [8]:
def save_dataframe_to_local(output_folder, file_name, df):
    """Guarda un DataFrame como archivo CSV en una carpeta local."""
    try:
        # Asegurarse de que la carpeta exista
        os.makedirs(output_folder, exist_ok=True)

        # Ruta completa del archivo
        file_path = os.path.join(output_folder, file_name)
        # limpia la columna "Entidad"
        df['Entidad'] = df['Entidad'].apply(clean_entidad_field)
        # limpia la columna diagnotico
        df['Diagnostico'] = df['Diagnostico'].apply(clean_special_characters)
        df = process_identificacion_column(df)
        # Renombrar columnas del DataFrame para que coincidan con la tabla en la base de datos
        df.rename(columns={
            'Fecha de Toma de Muestra': 'fecha_toma_muestra',
            'Fecha de Ingreso': 'fecha_ingreso',
            'Fecha de Informe': 'fecha_informe',
            'Entidad': 'entidad',
            'EPS': 'eps',
            'Servicio': 'servicio',
            'Muestra Remitida': 'muestra_remitida',
            'Descripcion Macroscopica': 'descripcion_macroscopica',
            'Descripcion Microscopica': 'descripcion_microscopica',
            'Diagnostico': 'diagnostico',
            'Comentario': 'comentario',
            'llavePaciente': 'id_paciente',
            'Archivo': 'archivo'
        }, inplace=True)
        
        # Agregar la columna 'fuente' con un valor por defecto (si es necesario)
        df['fuente'] = 'nombre_fuente'  # Cambia 'nombre_fuente' por el valor que corresponda

        # Guardar como CSV
        df.to_csv(file_path, index=False)
        print(f"Archivo {file_name} guardado exitosamente en {output_folder}.")
    except Exception as e:
        print(f"Error al guardar el archivo {file_name}: {e}")

In [9]:
def get_table_columns(table_name, connection_params):
    """
    Obtiene las columnas de una tabla en PostgreSQL.

    Args:
        table_name (str): Nombre de la tabla en PostgreSQL.
        connection_params (dict): Parámetros de conexión a PostgreSQL.

    Returns:
        list: Lista con los nombres de las columnas de la tabla.
    """
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()

        # Consultar los nombres de las columnas de la tabla
        query = f"""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_name = '{table_name}';
        """
        cursor.execute(query)
        columns = [row[0] for row in cursor.fetchall()]

        return columns

    except Exception as e:
        print(f"Error al obtener las columnas de la tabla {table_name}: {e}")
        return []
    finally:
        if conn:
            cursor.close()
            conn.close()

In [10]:
def insert_into_info_lab(df, connection_params):
    """
    Inserta los datos de un DataFrame en la tabla 'infolab' de PostgreSQL,
    considerando solo las columnas que coincidan entre los nombres del DataFrame y los de la tabla.

    Args:
        df (pd.DataFrame): DataFrame con los datos a insertar.
        connection_params (dict): Diccionario con los parámetros de conexión a PostgreSQL.
    """
    # Obtener las columnas de la tabla 'infolab'
    table_columns = get_table_columns('infolab', connection_params)
    
    if not table_columns:
        raise ValueError("No se pudieron obtener las columnas de la tabla 'infolab'.")

    # Seleccionar las columnas que coincidan entre el DataFrame y la tabla
    matching_columns = [col for col in df.columns if col in table_columns]
    
    if not matching_columns:
        raise ValueError("No hay columnas coincidentes entre el DataFrame y la tabla 'infolab'.")

    # Conexión a PostgreSQL
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()
        
        # Crear la consulta INSERT dinámica
        columns_str = ", ".join(matching_columns)
        placeholders_str = ", ".join(["%s"] * len(matching_columns))
        insert_query = f"INSERT INTO infolab ({columns_str}) VALUES ({placeholders_str})"

        # Preparar los datos para la inserción
        values = df[matching_columns].values.tolist()

        # Ejecutar la consulta
        cursor.executemany(insert_query, values)
        conn.commit()

        print(f"Se insertaron {cursor.rowcount} filas en la tabla 'infolab'.")
    
    except Exception as e:
        print(f"Error al insertar los datos: {e}")
        conn.rollback()
    finally:
        if conn:
            cursor.close()
            conn.close()

In [11]:
connection_params = {
    'dbname': 'udenar',
    'user': 'postgres',
    'password': '12345678',
    'host': 'localhost',
    'port': 5432
}

# Ejecutar el procesamiento y guardar los resultados
df = process_all_files_in_folder(LOCAL_FOLDER)
csv_file_name = "labjimenez.csv"
save_dataframe_to_local(OUTPUT_FOLDER, csv_file_name, df)
insert_into_info_lab(df, connection_params)

Procesando archivo: D:/Carlos/Udenar-Investigación/Tesis/Extraccion Egresos CANREC/Extaer lab/LABORATORIOS/LABORATORIOS/LAB JIMENEZ\Q02-23-87245177.pdf
Procesando archivo: D:/Carlos/Udenar-Investigación/Tesis/Extraccion Egresos CANREC/Extaer lab/LABORATORIOS/LABORATORIOS/LAB JIMENEZ\Q03-23-30706361.pdf
Procesando archivo: D:/Carlos/Udenar-Investigación/Tesis/Extraccion Egresos CANREC/Extaer lab/LABORATORIOS/LABORATORIOS/LAB JIMENEZ\Q04-23-12952760.pdf
Procesando archivo: D:/Carlos/Udenar-Investigación/Tesis/Extraccion Egresos CANREC/Extaer lab/LABORATORIOS/LABORATORIOS/LAB JIMENEZ\Q05-23-1086329350.pdf
Procesando archivo: D:/Carlos/Udenar-Investigación/Tesis/Extraccion Egresos CANREC/Extaer lab/LABORATORIOS/LABORATORIOS/LAB JIMENEZ\Q06-23-59821509.pdf
Archivo labjimenez.csv guardado exitosamente en D:/Carlos/Udenar-Investigación/Tesis/Extraccion Egresos CANREC/Extraccion_Laboratorios/lab_jimenez.
Se insertaron 5 filas en la tabla 'infolab'.
